In [ ]:
import requests
from requests.exceptions import HTTPError
from requests_jwt import JWTAuth
import websockets
import asyncio
import json
import plotly
from plotly.io import from_json, read_json
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import flapjack
from flapjack import Flapjack

In [ ]:
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username='tim', password='chicken')

In [ ]:
od = fj.query('signal', name='OD')
cfp = fj.query('signal', search='CFP')
study = fj.query('study')
chems = fj.query('chemical', name='')
media = fj.query('media', search='glucosa')
assay = fj.query('assay')
print(assay)

In [ ]:
chemicals = fj.query('chemical', pubchemid=12345)
chemicals

In [ ]:
fig = await fj.plot(assay=assay.id,
                    media=media.id, 
                    type='Mean Expression',
                    biomass_signal=od.id,
                    ref_signal=cfp.id,
                    #bg_correction=2,
                    #min_biomass=0.05,
                    #remove_data=False,
                    normalize='None', 
                    subplots='Signal', 
                    markers='Vector', 
                    plot='Mean')

In [ ]:
fig

In [ ]:
dna = fj.query('dna', sboluri='test')
dna

In [ ]:
vector = fj.query('vector', dnas=dna.id)
for d in vector.dnas:
    print(d)
    
    
print(vector.dtypes)
print(vector.dtypes['dnas'])
vector

In [ ]:
df = await fj.analysis(assay=[assay.id[0]],
                    media=media.id, 
                    type='Alpha',
                    biomass_signal=od.id,
                    ref_signal=cfp.id,
                    #bg_correction=2,
                    #min_biomass=0.05,
                    #remove_data=False
                      )


In [ ]:
g = df.groupby('Signal')
for signal,gg in g:
    gg.groupby('Vector').mean().plot.bar(y='Alpha', style='.', label=signal)

In [ ]:
dna = fj.query('dna', name='CcaT')
vector = fj.query('vector', dnas=dna.id)
chemical = fj.query('chemical', name='IPTG')
biomass_signal = fj.query('signal', name='OD')

df = await fj.analysis(vector=vector.id,
                 analyte=chemical.id,
                 type='Induction Curve',
                 function='Mean Expression',
                 biomass_signal=biomass_signal.id
                )

In [ ]:
len(df)

In [ ]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
data = df[df.Signal=='Venus'][df.Vector=='CcaT+TMA5']
z,std = flapjack.fit_curve(flapjack.hill, data, x='Concentration', y='Expression')

In [ ]:
print(z)
print(std)

In [ ]:
data.plot(x='Concentration', y='Expression', label='CcaT+TMA5', style='+', logx=True)
x = np.logspace(-6,-3,100)
y = flapjack.hill(x, *z)
plt.plot(x, y, 'k--')
plt.ylim([0,20000])